In [1]:
#install required packages
import sys
!pip install atlasopenmagic
%pip install "pyarrow>=20.0.0"
# from atlasopenmagic import install_from_environment
# install_from_environment()

Note: you may need to restart the kernel to use updated packages.


In [3]:
# !pip install awkward
# !pip install pyarrow

In [3]:
import awkward as ak 
import vector
import time
import uproot
import numpy as np # # for numerical calculations such as histogramming
import matplotlib.pyplot as plt # for plotting
from matplotlib.ticker import MaxNLocator, AutoMinorLocator # for minor ticks

sys.path.append('./backend')
#from GetInputMagic import get_samples_magic
from GetInput import get_samples
from RunAnalysisUproot import analysis
from PlotHistogramAk import plot_histogram

The string code for the available final-state collections:

* '2to4lep' - two to four leptons with at least 7 GeV of $p_T$ each
* '2muons' - at least two muons with at least 10 GeV of $p_T$
* 'GamGam' - at least two photons with at least 25 GeV of $p_T$ each
* 'exactly4lep' - exactly four leptons with at least 7 GeV of $p_T$

The string code for the available Monte Carlo simulation dataset:
* 'Zee'
* 'Zmumu'
* 'Ztautau'
* 'Wlepnu'
* 'ttbar'
* 'H'
* 'ZZllll'

Please use '+' to combine data sets. For example, if you want to combine the data for the physics processes of $Z\rightarrow\tau\tau$ and $ZZ^{*}\rightarrow llll$, write 'Ztautau+ZZllll'.

Please include '**Data**' in the key defined for the final-state collection and '**Signal**' for the physical processes simulated by the MC.

In [4]:
keys_input = [r'Background $Z\rightarrow ee$, $W\rightarrow l\nu$',
              r'Signal $Z\rightarrow\mu\mu$',
              'Data 2to4lep']

string_codes_input = ['Zee+Wlepnu', 
                      'Zmumu',
                      '2to4lep']

colors_input = ["m", # purple
                "g", # green
                "k"] # black

In [5]:
samples = get_samples(keys_input, string_codes_input, colors_input)
print(samples)

{'Background $Z\\rightarrow ee$, $W\\rightarrow l\\nu$': {'files': ['./backend/datasets/ODEO_FEB2025_v0_2to4lep_mc_700321.Sh_2211_Zee_maxHTpTV2_CFilterBVeto.2to4lep.root', './backend/datasets/ODEO_FEB2025_v0_2to4lep_mc_700320.Sh_2211_Zee_maxHTpTV2_BFilter.2to4lep.root', './backend/datasets/ODEO_FEB2025_v0_2to4lep_mc_700322.Sh_2211_Zee_maxHTpTV2_CVetoBVeto.2to4lep.root', './backend/datasets/ODEO_FEB2025_v0_2to4lep_mc_700338.Sh_2211_Wenu_maxHTpTV2_BFilter.2to4lep.root'], 'color': 'm'}, 'Signal $Z\\rightarrow\\mu\\mu$': {'files': ['./backend/datasets/ODEO_FEB2025_v0_2to4lep_mc_700323.Sh_2211_Zmumu_maxHTpTV2_BFilter.2to4lep.root', './backend/datasets/ODEO_FEB2025_v0_2to4lep_mc_700324.Sh_2211_Zmumu_maxHTpTV2_CFilterBVeto.2to4lep.root', './backend/datasets/ODEO_FEB2025_v0_2to4lep_mc_700325.Sh_2211_Zmumu_maxHTpTV2_CVetoBVeto.2to4lep.root'], 'color': 'g'}, 'Data 2to4lep': {'files': ['./backend/datasets/ODEO_FEB2025_v0_2to4lep_data15_periodD.2to4lep.root', './backend/datasets/ODEO_FEB2025_v0_2t

In [9]:
luminosity = 36.6
fraction = 1
# Define what variables are important to our analysis
variables = ["lep_n", "lep_pt", "lep_eta", "lep_phi", "lep_e", 
             #"lep_ptvarcone30", "lep_topoetcone20", 
             "lep_type", "lep_charge",
             #"lep_isLooseID", "lep_isMediumID", "lep_isTightID",
             #"lep_isLooseIso", "lep_isTightIso",
             "trigE", "trigM", "lep_isTrigMatched"]

In [10]:
# Note: first lepton in each event is [:, 0], 2nd lepton is [:, 1] etc
# Functions return bool. True means we should remove the event

# Function to cut on the number of leptons in each event
def cut_lep_n(lep_n, user_input):
    return (lep_n == user_input)

# Function to cut on the lepton type (based on type of first two lep_type)
# lep_type is a number signifying the lepton type (electron (11) or muon (13))
def cut_lep_type(lep_type, user_input):
    sum_lep_type = lep_type[:, 0] + lep_type[:, 1] # Sum of first two leptons' type in the event 
    return (sum_lep_type == user_input)

# Function to cut on the lepton charge (based on charge of first two lep_charge)
def cut_lep_charge(lep_charge, user_input):
    product_lep_charge = lep_charge[:, 0] * lep_charge[:, 1] # Product of first two leptons' charge in the event
    return (product_lep_charge == user_input)

# Function to cut on the lepton transverse momentum
def cut_lep_pt(lep_pt, index, lower_limit):
    return (lep_pt[:, index] > lower_limit) # Accept events with lepton pt higher than lower limit

# Function to cut on the isolation pt (based on first two leptons)
def cut_lep_ptvarcone30(lep_ptvarcone30, upper_limit):
    # Accept events with lep_ptvarcone30 in the range
    return (lep_ptvarcone30[:, 0] < upper_limit) & (lep_ptvarcone30[:, 1] < upper_limit)

# Function to cut on the isolation et (based on first two leptons)
def cut_lep_topoetcone20(lep_topoetcone20, upper_limit):
    # Accept events with lep_topoetcone20 in the range
    return (lep_topoetcone20[:, 0] < upper_limit) & (lep_topoetcone20[:, 1] < upper_limit)

# Function to accept events with at least one lepton is triggering
def cut_trig_match(lep_trigmatch): 
    return ak.sum(lep_trigmatch, axis=1) >= 1

# Function to accept events that has been selected by any of the single electron OR muon triggers
def cut_trig(trigE, trigM):
    return trigE | trigM

# # Function to filter events based on the identification and isolation criteria of all leptons in each event
# def ID_iso_cut(electron_isID, muon_isID, electron_isIso, muon_isIso, lep_type, lep_n): 
#     return (ak.sum(((lep_type == 13) & muon_isID & muon_isIso) | 
#                    ((lep_type == 11) & electron_isID & electron_isIso), axis=1) == lep_n)

# Function to keep events that have all leptons passed the identification and isolation criteria
def ID_iso_cut(lep_isID, lep_isIso, lep_n): 
    return (ak.sum(lep_isID & lep_isIso, axis=1) == lep_n)

# Function to calculate the invariant mass using four momentum (pt, eta, phi, energy)    
def calculate_inv_mass(lep_pt, lep_eta, lep_phi, lep_e):
    four_momentum = vector.zip({"pt": lep_pt, "eta": lep_eta, "phi": lep_phi, "E": lep_e})
    invariant_mass = (four_momentum[:, 0] + four_momentum[:, 1]).M

    return invariant_mass

def selection_cut(data):
    # Keep events that pass electron / muon trigger 
    data = data[cut_trig(data['trigE'], data['trigM'])]
    # Keep events where at least one lepton is triggering
    data = data[cut_trig_match(data['lep_isTrigMatched'])] 
    print("after trig " + str(len(data)))

    # Lepton cuts
    lep_n = data['lep_n']
    data = data[cut_lep_n(lep_n, 2)]
    print("after lep_n cut " + str(len(data)))
          
    lep_type = data['lep_type']
    data = data[cut_lep_type(lep_type, 26)]
    print("after type cut " + str(len(data)))
    
    lep_charge = data['lep_charge']
    data = data[cut_lep_charge(lep_charge, -1)]
    print("after charge cut " + str(len(data)))
    
    # Invariant Mass
    data['mass'] = calculate_inv_mass(data['lep_pt'], data['lep_eta'], data['lep_phi'], data['lep_e'])

    return data

In [11]:
# Save data for mass
all_data = analysis(luminosity, fraction, samples, selection_cut, variables, ['mass'])

Processing Background $Z\rightarrow ee$, $W\rightarrow l\nu$ samples
	./backend/datasets/ODEO_FEB2025_v0_2to4lep_mc_700321.Sh_2211_Zee_maxHTpTV2_CFilterBVeto.2to4lep.root:
after trig 1484347
after lep_n cut 1432672
after type cut 0
after charge cut 0
Data not in sample_key
		 nIn: 1610834,	 nOut: 	0	 in 5.8s
after trig 1485016
after lep_n cut 1433570
after type cut 0
after charge cut 0
Data not in sample_key
		 nIn: 1610834,	 nOut: 	0	 in 11.3s
after trig 1484685
after lep_n cut 1433149
after type cut 0
after charge cut 0
Data not in sample_key
		 nIn: 1610834,	 nOut: 	0	 in 16.8s
after trig 1484794
after lep_n cut 1433158
after type cut 0
after charge cut 0
Data not in sample_key
		 nIn: 1610834,	 nOut: 	0	 in 22.1s
after trig 1485054
after lep_n cut 1432992
after type cut 0
after charge cut 0
Data not in sample_key
		 nIn: 1610834,	 nOut: 	0	 in 27.6s
after trig 1484635
after lep_n cut 1433239
after type cut 0
after charge cut 0
Data not in sample_key
		 nIn: 1610834,	 nOut: 	0	 in 3

In [ ]:
all_data.keys()

In [ ]:
all_data['Signal $Z\\rightarrow\\mu\\mu$']

In [ ]:
import pickle

# Save all_data to file
with open("all_data.pkl", "wb") as f:
    pickle.dump(all_data, f)

In [ ]:
GeV = 1.0
# x-axis range of the plot
xmin = 0 * GeV
xmax = 120 * GeV

# Histogram bin setup
step_size = 0.2 * GeV

fig, ax = plot_histogram(all_data, 'mass',
                         xmin, xmax,
                         step_size,
                         '2-lepton invariant mass $\mathrm{m_{2l}}$ [GeV]',
                         marker='.', label_fontsize=16, title='')


In [ ]:
fig.savefig('histogram.png')